# Exponential Smoothing

In [72]:
from datetime import datetime
import pandas as pd
from bokeh.plotting import figure, show 
from bokeh.io import output_notebook, curdoc
from bokeh.layouts import layout, column, row
from bokeh.models import DatetimeTickFormatter, HoverTool, Band, CDSView, ColumnDataSource, IndexFilter, FactorRange, Whisker, TableColumn, DataTable, LinearAxis, Range1d
from bokeh.palettes import Dark2

output_notebook()


Loading BokehJS ...

## Texas Oil and Gas Production
Texas Statewide monthly data for oil and gas production from Jan 1993 to Aug 2021 taken from the Texas Railroad Commission's [Production Data Query System](http://webapps.rrc.state.tx.us/PDQ/home.do).

In [47]:
col_dtypes={
    'Oil (BBL)': "Int64",
    'Casinghead (MCF)': "Int64",
    'GW Gas (MCF)': "Int64",
    'Condensate (BBL)': "Int64"
}
df = pd.read_csv("oil and gas producion texas.csv", thousands=",")
df["Date"] = pd.to_datetime(df["Date"], format="%b-%y")
df = df.set_index('Date')
df.head()

,Oil (BBL),Casinghead (MCF),GW Gas (MCF),Condensate (BBL)
Date,,,,
1993-01-01,50411018,109239999,368487532,2868768
1993-02-01,45634703,100315311,333803989,2635625
1993-03-01,50288158,112470507,370855741,2912724
1993-04-01,48194498,108833718,357648783,2829663
1993-05-01,49330060,111948168,364968117,2961272


### Definitions
Taken from the Texas Railroad Commission Website [Glossary of Terms](http://webapps.rrc.state.tx.us/PDQ/help/prod_gloss.html#head_C)

**Oil Well**<br>
Any well which produces one barrel or more crude petroleum oil to each 100,000 cubic feet of natural gas.

**Casinghead Gas**<br>
Gas found naturally in oil and produced with the oil.

**Gas Well**<br>
Any well:
1. which produces natural gas not associated or blended with crude petroleum oil at the time of production;
2. which produces more than 100,000 cubic feet of natural gas to each barrel of crude petroleum oil from the same producing horizon; or
3. which produces natural gas from a formation or producing horizon productive of gas only encountered in a wellbore through which crude petroleum oil also is produced through the inside of another string of casing or tubing. A well which produces hydrocarbon liquids, a part of which is formed by a condensation from a gas phase and a part of which is crude petroleum oil, shall be classified as a gas well unless there is produced one barrel or more of crude petroleum oil per 100,000 cubic feet of natural gas; and that the term "crude petroleum oil" shall not be construed to mean any liquid hydrocarbon mixture or portion thereof which is not in the liquid phase in the reservoir, removed from the reservoir in such liquid phase, and obtained at the surface as such.

**Condensate**<br>
Any liquid hydrocarbon recovered by surface separators from natural gas.

**MCF**<br>
One thousand cubic feet of natural gas measured at standard pressure and temperature conditions (see "cubic foot of gas"). The term is commonly used to express the volume of gas produced, transmitted or consumed.

**BBL** <br>
Abbreviation for barrel. In the energy industry, a barrel is 42 U.S. gallons measured at 60 ° Fahrenheit.

In [78]:
source = ColumnDataSource((df/1000000).reset_index())
bbl_cols = ["Oil (BBL)", "Condensate (BBL)"]
mcf_cols = ["Casinghead (MCF)", "GW Gas (MCF)"]
p = figure(
    title="Texas Statewide Oil and Gas Production (Monthly)", 
    y_axis_label="BBL (millions)", 
    y_range = (0,df[bbl_cols].max().max()/1000000),
    sizing_mode="stretch_width",
    height=400
    )
p.line(x="Date", y="Oil (BBL)", legend_label="Oil", source = source, line_color=Dark2[4][0], line_width=2)
p.line(x="Date", y="Condensate (BBL)", legend_label="Condensate", source = source, line_color=Dark2[4][1], line_width=2)
p.extra_y_ranges = {"mcf": Range1d(start=0, end=df[mcf_cols].max().max()/1000000)}
p.add_layout(LinearAxis(y_range_name="mcf", axis_label="MCF (millions)"), 'right')
p.line(x="Date", y="Casinghead (MCF)", legend_label="Casinghead", source = source, y_range_name="mcf", line_color=Dark2[4][2], line_width=2)
p.line(x="Date", y="GW Gas (MCF)", legend_label="GW Gas", source = source, y_range_name="mcf", line_color=Dark2[4][3], line_width=2)
p.xaxis.formatter = DatetimeTickFormatter(
    days = ['%b %Y'],
    months = ['%b %Y'],
    years = ['%Y']
    )
p.add_tools(HoverTool(
    tooltips="@Date{%b %d, %Y} $y",
    formatters={'@Date': 'datetime'},
    mode='vline'
))
p.legend.location = "top_left"
show(p)

In [60]:
df[["Oil (BBL)", "Condensate (BBL)"]].max().max()

142698592